In [44]:
import sys
import os
sys.path.append(os.getcwd())

In [45]:
import sys

# sys.modules에서 기본 모듈을 제외하고 사용자가 임포트한 모듈만 삭제
modules_to_delete = [mod for mod in sys.modules if mod.startswith('KIS_') or mod.startswith('my_custom_module')]

for module in modules_to_delete:
    # print(module)
    del sys.modules[module]


In [46]:
import KIS_Common as Common
import pandas as pd
import KIS_API_Helper_US as KisUS
import time
import json
import pprint
from datetime import datetime, timedelta

import line_alert

In [38]:
import importlib
import KIS_API_Helper_US
importlib.reload(KIS_API_Helper_US)


<module 'KIS_API_Helper_US' from 'x:\\자동매매\\게만아\\49.스노우미국형전략_UPRO_분석\\KIS_API_Helper_US.py'>

In [47]:
#계좌 선택.. "VIRTUAL" 는 모의 계좌!
Common.SetChangeMode("VIRTUAL") #REAL or VIRTUAL

BOT_NAME = Common.GetNowDist() + "_MySnowNBotUS"

#시간 정보를 읽는다
time_info = time.gmtime()
#년월 문자열을 만든다 즉 2022년 9월이라면 2022_9 라는 문자열이 만들어져 strYM에 들어간다!
strYM = str(time_info.tm_year) + "_" + str(time_info.tm_mon)
print("ym_st: " , strYM)

#포트폴리오 이름
PortfolioName = "Snow전략_US"


ym_st:  2024_10


In [48]:
#리밸런싱이 가능한지 여부를 판단!
Is_Rebalance_Go = False

#파일에 저장된 년월 문자열 (ex> 2022_9)를 읽는다
YMDict = dict()

#파일 경로입니다.
asset_tym_file_path = BOT_NAME + ".json"

print(asset_tym_file_path)

try:
    with open(asset_tym_file_path, 'r') as json_file:
        YMDict = json.load(json_file)

except Exception as e:
    print("Exception by First")
    
print(YMDict)


VIRTUAL_MySnowNBotUS.json
{'ym_st': '2024_10'}


In [49]:
#만약 키가 존재 하지 않는다 즉 아직 한번도 매매가 안된 상태라면
if YMDict.get("ym_st") == None:

    #리밸런싱 가능! (리밸런싱이라기보다 첫 매수해야 되는 상황!)
    Is_Rebalance_Go = True
    
#매매가 된 상태라면! 매매 당시 혹은 리밸런싱 당시 년월 정보(ex> 2022_9) 가 들어가 있다.
else:
    #그럼 그 정보랑 다를때만 즉 달이 바뀌었을 때만 리밸런싱을 해야 된다
    if YMDict['ym_st'] != strYM:
        #리밸런싱 가능!
        Is_Rebalance_Go = True
        
print(strYM)

2024_10


In [50]:
#마켓이 열렸는지 여부~!
IsMarketOpen = KisUS.IsMarketOpen()

if IsMarketOpen == True:
    print("Market Is Open!!!!!!!!!!!")
    #영상엔 없지만 리밸런싱이 가능할때만 내게 메시지를 보내자!
    # if Is_Rebalance_Go == True:
    #     line_alert.SendMessage(PortfolioName + " (" + strYM + ") 장이 열려서 포트폴리오 리밸런싱 가능!!")
else:
    print("Market Is Close!!!!!!!!!!!")
    #영상엔 없지만 리밸런싱이 가능할때만 내게 메시지를 보내자!
    # if Is_Rebalance_Go == True:
    #     line_alert.SendMessage(PortfolioName + " (" + strYM + ") 장이 닫혀서 포트폴리오 리밸런싱 불가능!!")


datetime.datetime(2024, 10, 8, 18, 59, 21, 449635, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)
Time is NO!!!
Market Is Close!!!!!!!!!!!


In [52]:
#계좌 잔고를 가지고 온다!
Balance = KisUS.GetBalance()

print("--------------내 보유 잔고---------------------")
pprint.pprint(Balance)
print("--------------------------------------------")

#총 평가금액에서 해당 봇에게 할당할 총 금액비율 1.0 = 100%  0.5 = 50%
InvestRate = 0.05

#기준이 되는 내 총 평가금액에서 투자비중을 곱해서 나온 포트폴리오에 할당된 돈!!
TotalMoney = float(Balance['TotalMoney']) * InvestRate

print("총 포트폴리오에 할당된 투자 가능 금액 : $", str(round(TotalMoney,2),','))

--------------내 보유 잔고---------------------
{'RemainMoney': 209395.14444857035,
 'StockMoney': 45446.51,
 'StockRevenue': 295.3400000000038,
 'TotalMoney': 254841.65444857036}
--------------------------------------------
총 포트폴리오에 할당된 투자 가능 금액 : $ 12742.08


In [54]:
TotalMoney = 254841.65444857036 *0.05
print("총 포트폴리오에 할당된 투자 가능 금액 : $", str(round(TotalMoney,2)))

# 세자리씩 , 구분 필요

총 포트폴리오에 할당된 투자 가능 금액 : $ 12742.08


In [61]:
#투자 주식 리스트
MyPortfolioList = list()

StockCodeList = ["TIP","UPRO","TYD","VNQ","SPTL","VGIT","PFIX","RISR","USHY","VCIT","VWOB","BNDX","BWX","SGOV","PDBC"]

for stock_code in StockCodeList:
    asset = dict()
    asset['stock_code'] = stock_code         #종목코드
    asset['stock_target_rate'] = 0     #비중..
    asset['stock_rebalance_amt'] = 0  #리밸런싱 수량
    MyPortfolioList.append(asset)


In [62]:
print("--------------내 보유 주식---------------------")
#그리고 현재 이 계좌에서 보유한 주식 리스트를 가지고 옵니다!
MyStockList = KisUS.GetMyStockList()
df0 = pd.DataFrame(MyStockList)
pprint.pprint(MyStockList)

--------------내 보유 주식---------------------
[{'StockAmt': '218',
  'StockAvgPrice': '46.7060',
  'StockCode': 'EEM',
  'StockName': 'ISHARES MSCI EMERGING FUND',
  'StockNowMoney': '10069.42000000',
  'StockNowPrice': '46.190000',
  'StockOriMoney': '10182.10000',
  'StockRevenueMoney': '-112.680000',
  'StockRevenueRate': '-1.11'},
 {'StockAmt': '12',
  'StockAvgPrice': '244.6750',
  'StockCode': 'GLD',
  'StockName': 'SPDR GOLD TRUST',
  'StockNowMoney': '2908.44000000',
  'StockNowPrice': '242.370000',
  'StockOriMoney': '2936.10000',
  'StockRevenueMoney': '-27.660000',
  'StockRevenueRate': '-0.94'},
 {'StockAmt': '13',
  'StockAvgPrice': '486.0500',
  'StockCode': 'QQQ',
  'StockName': 'INVESCO QQQ TRUST UNIT SER 1',
  'StockNowMoney': '6360.90000000',
  'StockNowPrice': '489.300000',
  'StockOriMoney': '6318.65000',
  'StockRevenueMoney': '42.250000',
  'StockRevenueRate': '0.67'},
 {'StockAmt': '30',
  'StockAvgPrice': '84.2800',
  'StockCode': 'SCHD',
  'StockName': 'SCHWAB US 

In [60]:
df0   # 보유 주식 파일로 저장 후 가공해서 line 메세지 공유유 필요

,StockCode,StockName,StockAmt,StockAvgPrice,StockOriMoney,StockNowMoney,StockNowPrice,StockRevenueMoney,StockRevenueRate
0,EEM,ISHARES MSCI EMERGING FUND,218,46.7060,10182.10000,10069.42000000,46.190000,-112.680000,-1.11
1,GLD,SPDR GOLD TRUST,12,244.6750,2936.10000,2908.44000000,242.370000,-27.660000,-0.94
2,QQQ,INVESCO QQQ TRUST UNIT SER 1,13,486.0500,6318.65000,6360.90000000,489.300000,42.250000,0.67
3,SCHD,SCHWAB US DIVIDEND EQUITY ETF,30,84.2800,2528.40000,2524.80000000,84.160000,-3.600000,-0.14
4,TLT,ISHARES 20+Y TREASURY BOND ETF,3,94.9500,284.85000,284.97000000,94.990000,0.120000,0.04
5,TMF,DIREXION DAILY 20Y TREASURY BULL 3X ETF,70,54.1200,3788.40000,3689.00000000,52.700000,-99.400000,-2.62
6,TQQQ,PROETF ULTRAPRO QQQ,91,70.0000,6370.00000,6608.42000000,72.620000,238.420000,3.74
7,UPRO,PROETF ULTRAPRO S&P 500,152,83.8330,12742.67000,13000.56000000,85.530000,257.890000,2.02


In [ ]:
stock_df_list = []

for stock_code in StockCodeList:
    # print(stock_code)
  
    df = Common.GetOhlcv("US", stock_code,300) 

    df['prevClose'] = df['close'].shift(1)

    df['1month'] = df['close'].shift(20)
    df['3month'] = df['close'].shift(60)
    df['5month'] = df['close'].shift(100)
    df['6month'] = df['close'].shift(120)
    df['9month'] = df['close'].shift(180)
    df['12month'] = df['close'].shift(240)

    #1개월 수익률 + 3개월 수익률 + 6개월 수익률 + 9개월 수익률 + 12개월 수익률
    df['Momentum'] = ( ((df['prevClose'] - df['1month'])/df['1month']) + ((df['prevClose'] - df['3month'])/df['3month']) + ((df['prevClose'] - df['6month'])/df['6month']) + ((df['prevClose'] - df['9month'])/df['6month'])  + ((df['prevClose'] - df['12month'])/df['12month']) ) / 5.0
    
    #1개월 수익률 x 12 + 3개월 수익률 x 4 + 6개월 수익률 x 2 + 12개월 수익률
    df['MomentumGA'] = (((df['prevClose'] - df['1month'])/df['1month']) * 12.0) + (((df['prevClose'] - df['3month'])/df['3month']) * 4.0) + (((df['prevClose'] - df['6month'])/df['6month']) *2.0 ) + ((df['prevClose'] - df['12month'])/df['12month'])
    
    #6개월 모멘텀
    df['Momentum_6'] = (df['prevClose'] - df['6month'])/df['6month']

    df.dropna(inplace=True) #데이터 없는건 날린다!

    data_dict = {stock_code: df}
    stock_df_list.append(data_dict)
        
    print("---stock_code---", stock_code , " len ",len(df))
    pprint.pprint(df)

In [73]:
for stock_code in StockCodeList:
    print(stock_code)

TIP
UPRO
TYD
VNQ
SPTL
VGIT
PFIX
RISR
USHY
VCIT
VWOB
BNDX
BWX
SGOV
PDBC


In [65]:
stock_df_list

[{'TIP':                   open        high         low       close     volume  \
  Date                                                                    
  2024-07-16  107.589996  107.860001  107.529999  107.449203   818200.0   
  2024-07-17  107.750000  107.900002  107.650002  107.489090   956000.0   
  2024-07-18  107.680000  107.830002  107.559998  107.249802  1272900.0   
  2024-07-19  107.459999  107.480003  107.290001  106.990585  1948200.0   
  2024-07-22  107.290001  107.349998  107.010002  106.860970  2347900.0   
  2024-07-23  107.199997  107.349998  107.110001  106.801155  2039600.0   
  2024-07-24  107.230003  107.339996  106.800003  106.482101  1252900.0   
  2024-07-25  106.940002  107.239998  106.930000  106.741325  1073200.0   
  2024-07-26  107.360001  107.440002  107.230003  107.080315  1458800.0   
  2024-07-29  107.589996  107.599998  107.419998  107.189995   973300.0   
  2024-07-30  107.610001  107.709999  107.449997  107.269745   693500.0   
  2024-07-31  107.

In [81]:
# print(data_dict.values())
print([list(data_dict.values())[0].assign(stock_code=stock_code) for data_dict in stock_df_list for stock_code in data_dict])

[                  open        high         low       close     volume  \
Date                                                                    
2024-07-16  107.589996  107.860001  107.529999  107.449203   818200.0   
2024-07-17  107.750000  107.900002  107.650002  107.489090   956000.0   
2024-07-18  107.680000  107.830002  107.559998  107.249802  1272900.0   
2024-07-19  107.459999  107.480003  107.290001  106.990585  1948200.0   
2024-07-22  107.290001  107.349998  107.010002  106.860970  2347900.0   
2024-07-23  107.199997  107.349998  107.110001  106.801155  2039600.0   
2024-07-24  107.230003  107.339996  106.800003  106.482101  1252900.0   
2024-07-25  106.940002  107.239998  106.930000  106.741325  1073200.0   
2024-07-26  107.360001  107.440002  107.230003  107.080315  1458800.0   
2024-07-29  107.589996  107.599998  107.419998  107.189995   973300.0   
2024-07-30  107.610001  107.709999  107.449997  107.269745   693500.0   
2024-07-31  107.889999  108.199997  107.639999  10

In [82]:
combined_df = pd.concat([list(data_dict.values())[0].assign(stock_code=stock_code) for data_dict in stock_df_list for stock_code in data_dict])
combined_df.sort_index(inplace=True)
pprint.pprint(combined_df['MomentumGA'])
print(" len(combined_df) ", len(combined_df))

Date
2024-07-16    0.342548
2024-07-16    1.165348
2024-07-16    0.417438
2024-07-16    0.201571
2024-07-16    0.283696
                ...   
2024-10-08    3.104063
2024-10-08    0.167044
2024-10-08    0.216658
2024-10-08    0.300901
2024-10-08    1.569336
Name: MomentumGA, Length: 900, dtype: float64
 len(combined_df)  900


In [76]:
combined_df.describe

<bound method NDFrame.describe of                   open        high         low       close     volume  \
Date                                                                    
2024-07-16  107.589996  107.860001  107.529999  107.449203   818200.0   
2024-07-16   47.680000   47.680000   46.299999   45.788960    41300.0   
2024-07-16   81.070000   81.169998   80.919998   80.281082  4998200.0   
2024-07-16  100.519997  100.519997  100.510002   99.201530  3622900.0   
2024-07-16   25.910000   26.090000   25.799999   25.866390    47700.0   
...                ...         ...         ...         ...        ...   
2024-10-08   84.300003   85.779800   83.930000   85.529999  3004260.0   
2024-10-08   22.730000   22.688000   22.600000   22.650000   146154.0   
2024-10-08  109.080002  109.470001  109.040001  109.449997  1951033.0   
2024-10-08   82.555901   82.629997   82.364998   82.629997  5529140.0   
2024-10-08   13.960000   13.980000   13.750000   13.880000  7072744.0   

                

In [67]:
date = combined_df.iloc[-1].name

tip_data = combined_df[(combined_df.index == date) & (combined_df['stock_code'] == "TIP")] 
pprint.pprint(tip_data)

                  open        high         low       close     volume  \
Date                                                                    
2024-10-08  109.080002  109.470001  109.040001  109.449997  1951033.0   

                   value    change   prevClose      1month      3month  \
Date                                                                     
2024-10-08  2.131699e+08  0.001647  109.269997  109.858543  107.120201   

                5month      6month      9month     12month  Momentum  \
Date                                                                   
2024-10-08  105.353363  103.613243  104.833176  100.111794  0.040721   

            MomentumGA  Momentum_6 stock_code  
Date                                           
2024-10-08    0.216658    0.054595        TIP  


In [83]:
#안전자산을 제외한 공격자산의 모멘텀 높은거 1개 픽!!!!
exclude_stock_codes = ["TIP","SPTL","VGIT","PFIX","RISR","USHY","VCIT","VWOB","BNDX","BWX","SGOV"] #공격자산에 포함되면 안되는 종목들 즉 안전자산들을 넣으면 된다!
pick_momentum_top = combined_df.loc[(combined_df.index == date)  & (~combined_df['stock_code'].isin(exclude_stock_codes)) ].groupby('stock_code')['MomentumGA'].max().nlargest(10)
pprint.pprint(pick_momentum_top)

stock_code
UPRO    3.104063
PDBC    1.569336
VNQ     0.832836
TYD    -0.362419
Name: MomentumGA, dtype: float64


In [84]:
#공격자산을 제외한 안전자산 모멘텀 스코어가 높은거 상위 TOP 3개를 리턴!
exclude_stock_codes = ["UPRO","TYD","VNQ","PDBC"] #안전자산에 포함되면 안되는 종목들 즉 공격자산들을 넣으면 된다!
pick_bond_momentum_top = combined_df.loc[(combined_df.index == date) & (~combined_df['stock_code'].isin(exclude_stock_codes))].groupby('stock_code')['Momentum_6'].max().nlargest(4)
pprint.pprint(pick_bond_momentum_top)

stock_code
SPTL    0.085240
VWOB    0.081919
USHY    0.079146
VCIT    0.076402
Name: Momentum_6, dtype: float64


In [70]:
if len(tip_data) == 1 :
    #안전 자산 비중 정하기!
    SafeRate = 0
    AtkRate = 0
        
    AtkOkList = list()

    #TIP 모멘텀 양수 장이 좋다!
    if tip_data['Momentum'].values[0] > 0 :
        
        #공격자산에 100% 투자한다!
        AtkRate = 1.0
        for stock_code in pick_momentum_top.index:
            stock_data = combined_df[(combined_df.index == date) & (combined_df['stock_code'] == stock_code)]

            if len(stock_data) == 1:
                AtkOkList.append(stock_code)

    #TIP 모멘텀 음수 장이 안좋아!
    else:
        #안전자산에 100% 투자한다!
        SafeRate = 1.0

    #리밸런싱 수량을 확정한다!
    for stock_info in MyPortfolioList:
        stock_code = stock_info['stock_code']
        stock_data = combined_df[(combined_df.index == date) & (combined_df['stock_code'] == stock_code)] 
        
        if len(stock_data) == 1:  
            IsRebalanceGo = False
            NowClosePrice = stock_data['close'].values[0]

            #안전 자산 비중이 있는 경우!! 안전자산에 투자!!!
            if SafeRate > 0:
                for stock_code_b in pick_bond_momentum_top.index:   
                    if stock_code_b == stock_code:
                        if stock_data['Momentum_6'].values[0] > 0 :
                            stock_info['stock_target_rate'] += (SafeRate/len(pick_bond_momentum_top.index))
                            print("방어자산! ", stock_code)
                        break

            #선택된 공격자산이라면!!
            if stock_code in AtkOkList:
                stock_info['stock_target_rate'] +=  (AtkRate/len(pick_momentum_top.index))
                print("공격자산! ", stock_code)

공격자산!  UPRO


In [71]:
strResult = "-- 현재 포트폴리오 상황 --\n"

#매수된 자산의 총합!
total_stock_money = 0


In [72]:
#현재 평가금액 기준으로 각 자산이 몇 주씩 매수해야 되는지 계산한다 (포트폴리오 비중에 따라) 이게 바로 리밸런싱 목표치가 됩니다.
for stock_info in MyPortfolioList:

    #내주식 코드
    stock_code = stock_info['stock_code']
    #매수할 자산 보유할 자산의 비중을 넣어준다!
    stock_target_rate = float(stock_info['stock_target_rate']) 

    #현재가!
    CurrentPrice = KisUS.GetCurrentPrice(stock_code)
    
    stock_name = ""
    stock_amt = 0 #수량
    stock_avg_price = 0 #평단
    stock_eval_totalmoney = 0 #총평가금액!
    stock_revenue_rate = 0 #종목 수익률
    stock_revenue_money = 0 #종목 수익금

    #내가 보유한 주식 리스트에서 매수된 잔고 정보를 가져온다
    for my_stock in MyStockList:
        if my_stock['StockCode'] == stock_code:
            stock_name = my_stock['StockName']
            stock_amt = int(my_stock['StockAmt'])
            stock_avg_price = float(my_stock['StockAvgPrice'])
            stock_eval_totalmoney = float(my_stock['StockNowMoney'])
            stock_revenue_rate = float(my_stock['StockRevenueRate'])
            stock_revenue_money = float(my_stock['StockRevenueMoney'])
            break

    print("##### stock_code: ", stock_code)

    #주식의 총 평가금액을 더해준다
    total_stock_money += stock_eval_totalmoney

    #현재 비중
    stock_now_rate = 0

    #잔고에 있는 경우 즉 이미 매수된 주식의 경우
    if stock_amt > 0:
        stock_now_rate = round((stock_eval_totalmoney / TotalMoney),3)
        print("---> NowRate:", round(stock_now_rate * 100.0,2), "%")

        if stock_target_rate == 0:
            stock_info['stock_rebalance_amt'] = -stock_amt
            print("!!!!!!!!! SELL")
            
        else:
            #목표한 비중이 다르다면!!
            if stock_now_rate != stock_target_rate:

                #갭을 구한다!!!
                GapRate = stock_target_rate - stock_now_rate

                #그래서 그 갭만큼의 금액을 구한다
                GapMoney = TotalMoney * abs(GapRate) 
                #현재가로 나눠서 몇주를 매매해야 되는지 계산한다
                GapAmt = GapMoney / CurrentPrice

                #수량이 1보다 커야 리밸러싱을 할 수 있다!! 즉 그 전에는 리밸런싱 불가 
                if GapAmt >= 1.0:
                    GapAmt = int(GapAmt)
                    #갭이 음수라면! 비중이 더 많으니 팔아야 되는 상황!!! 
                    if GapRate < 0:
                        print("this!!!")
                        stock_info['stock_rebalance_amt'] = -GapAmt

                    #갭이 양수라면 비중이 더 적으니 사야되는 상황!
                    else:  
                        stock_info['stock_rebalance_amt'] = GapAmt

    #잔고에 없는 경우
    else:
        print("---> NowRate: 0%")
        if stock_target_rate > 0:
            #잔고가 없다면 첫 매수다! 비중대로 매수할 총 금액을 계산한다 
            BuyMoney = TotalMoney * stock_target_rate

            #매수할 수량을 계산한다!
            BuyAmt = int(BuyMoney / CurrentPrice)

            stock_info['stock_rebalance_amt'] = BuyAmt
        
    #라인 메시지랑 로그를 만들기 위한 문자열 
    line_data =  (">> " + stock_code + " << \n비중: " + str(round(stock_now_rate * 100.0,2)) + "/" + str(round(stock_target_rate * 100.0,2)) 
    + "% \n수익: $" + str(stock_revenue_money) + "("+ str(round(stock_revenue_rate,2)) 
    + "%) \n총평가금액: $" + str(round(stock_eval_totalmoney,2)) 
    + "\n현재보유수량: " + str(stock_amt) 
    + "\n리밸런싱수량: " + str(stock_info['stock_rebalance_amt']) + "\n----------------------\n")

    if Is_Rebalance_Go == True:
        line_alert.SendMessage(line_data)
    strResult += line_data

print("--------------리밸런싱 해야 되는 수량-------------")
data_str = "\n" + PortfolioName + "\n" +  strResult + "\n포트폴리오할당금액: $" + str(round(TotalMoney,2)) + "\n매수한자산총액: $" + str(round(total_stock_money,2))

#결과를 출력해 줍니다!
print(data_str)
#영상엔 없지만 리밸런싱이 가능할때만 내게 메시지를 보내자!
#if Is_Rebalance_Go == True:
#    line_alert.SendMessage(data_str)

##### stock_code:  TIP
---> NowRate: 0%
##### stock_code:  UPRO
---> NowRate: 102.0 %
this!!!
##### stock_code:  TYD
---> NowRate: 0%
##### stock_code:  VNQ
---> NowRate: 0%
##### stock_code:  SPTL
---> NowRate: 0%
##### stock_code:  VGIT
---> NowRate: 0%
##### stock_code:  PFIX
---> NowRate: 0%
##### stock_code:  RISR
---> NowRate: 0%
##### stock_code:  USHY
---> NowRate: 0%
##### stock_code:  VCIT
---> NowRate: 0%
##### stock_code:  VWOB
---> NowRate: 0%
##### stock_code:  BNDX
---> NowRate: 0%
##### stock_code:  BWX
---> NowRate: 0%
##### stock_code:  SGOV
---> NowRate: 0%
##### stock_code:  PDBC
---> NowRate: 0%
--------------리밸런싱 해야 되는 수량-------------

Snow전략_US
-- 현재 포트폴리오 상황 --
>> TIP << 
비중: 0.0/0.0% 
수익: $0(0%) 
총평가금액: $0
현재보유수량: 0
리밸런싱수량: 0
----------------------
>> UPRO << 
비중: 102.0/100.0% 
수익: $257.89(2.02%) 
총평가금액: $13000.56
현재보유수량: 152
리밸런싱수량: -2
----------------------
>> TYD << 
비중: 0.0/0.0% 
수익: $0(0%) 
총평가금액: $0
현재보유수량: 0
리밸런싱수량: 0
----------------------
>> VNQ << 
비중